# RINEX Processing – PACQUI

This notebook processes RINEX `.25O` files for the **PACQUI** site.

It performs three main steps:
1. Defines paths and configuration parameters.
2. Extracts ZIP files and consolidates all `.25O` files into one folder.
3. Converts RINEX files into NetCDF format using the `gnssvod` library.

All logic remains identical to the original Spanish version; only the text and structure are translated and organized for clarity.

## 🧭 1. Path Configuration
Defines input/output paths and parameters used by the workflow.
- `ZIP_ROOT`: folder containing PACQUI ZIP files.
- `DEST_ROOT`: destination folder for consolidated `.25O` files.
- `UNZIP_ROOT`: temporary extraction folder.
- `RINEX_SUBSTR`: identifier substring for RINEX directories.
- `ZIP_PATTERN`: filename pattern for PACQUI ZIPs.
- `OVERWRITE_25O`: overwrite policy for `.25O` files.

In [1]:
# === PATH CONFIGURATION (PACQUI) ===
from pathlib import Path

# Folder containing PACQUI .zip files
ZIP_ROOT = Path(r"C:\Users\varga\Box\Data\RinexZipFiles\PACQUI\Last")

# Single destination folder for consolidated .25O files
DEST_ROOT = Path(r"C:\Users\varga\Box\Data\nc Files\PACQUI")

# Substring that identifies the RINEX folder name
RINEX_SUBSTR = "RINEX_3_03"

# ZIP filename pattern for PACQUI
ZIP_PATTERN = "PACQUI*.zip"

# Temporary unzip folder (created next to ZIP_ROOT)
UNZIP_ROOT = ZIP_ROOT / "unzipped"

# Overwrite policy for .25O files
OVERWRITE_25O = False

ZIP_ROOT, DEST_ROOT, UNZIP_ROOT

(WindowsPath('C:/Users/varga/Box/Data/RinexZipFiles/PACQUI/Last'),
 WindowsPath('C:/Users/varga/Box/Data/nc Files/PACQUI'),
 WindowsPath('C:/Users/varga/Box/Data/RinexZipFiles/PACQUI/Last/unzipped'))

## 📦 2. Extract ZIPs and Copy All `.25O` Files
Extracts all PACQUI ZIP files, finds RINEX directories, and copies every `.25O` file into a single flat folder under `DEST_ROOT`. Logic identical to original.

In [2]:
# === LOGIC: extract zips and copy ALL *.25O into a single folder (PACQUI) ===
import zipfile, shutil
from pathlib import Path

def safe_extract(zf: zipfile.ZipFile, dest: Path):
    for member in zf.infolist():
        member_path = dest / member.filename
        if not str(member_path.resolve()).startswith(str(dest.resolve())):
            raise RuntimeError(f"Unsafe ZIP entry: {member.filename}")
        zf.extract(member, dest)

def extract_all_zips(root: Path, unzip_root: Path, pattern: str):
    unzip_root.mkdir(parents=True, exist_ok=True)
    zips = sorted(root.glob(pattern), key=lambda p: p.name.lower())
    extracted = []
    for z in zips:
        target_dir = unzip_root / z.stem
        if target_dir.exists():
            extracted.append(target_dir)
            continue
        try:
            with zipfile.ZipFile(z) as zf:
                print(f"[EXTRACT] {z.name} -> {target_dir.name}")
                target_dir.mkdir(parents=True, exist_ok=True)
                safe_extract(zf, target_dir)
                extracted.append(target_dir)
        except Exception as e:
            print(f"[ERROR] {z.name}: {e}")
    return extracted

def find_rinex_dirs(base: Path, substr: str):
    sub = substr.lower()
    return [p for p in base.rglob("*") if p.is_dir() and sub in p.name.lower()]

def copy_25O_flat(unzip_root: Path, dest_root: Path, rinex_substr: str, overwrite=False):
    dest_root.mkdir(parents=True, exist_ok=True)
    total = 0
    for station_dir in sorted(p for p in unzip_root.iterdir() if p.is_dir()):
        rinex_dirs = find_rinex_dirs(station_dir, rinex_substr)
        if rinex_dirs:
            for rdir in rinex_dirs:
                for f in rdir.glob("*.25O"):
                    df = dest_root / f.name
                    if df.exists() and not overwrite:
                        continue
                    shutil.copy2(f, df)
                    total += 1
        else:
            for f in station_dir.rglob("*.25O"):
                df = dest_root / f.name
                if df.exists() and not overwrite:
                    continue
                shutil.copy2(f, df)
                total += 1
    return total

# === Run extraction ===
DEST_ROOT.mkdir(parents=True, exist_ok=True)
extracted_dirs = extract_all_zips(ZIP_ROOT, UNZIP_ROOT, ZIP_PATTERN)
for p in sorted(UNZIP_ROOT.iterdir() if UNZIP_ROOT.exists() else []):
    if p.is_dir() and p not in extracted_dirs:
        extracted_dirs.append(p)

total_copied = copy_25O_flat(UNZIP_ROOT, DEST_ROOT, RINEX_SUBSTR, overwrite=OVERWRITE_25O)
print("\n===== SUMMARY =====")
print(f"ZIPs processed/existing: {len(extracted_dirs)}")
print(f"Total *.25O files copied: {total_copied}")
print(f"Destination: {DEST_ROOT}")

[EXTRACT] PACQUI_20250418220206.zip -> PACQUI_20250418220206
[EXTRACT] PACQUI_20250419220207.zip -> PACQUI_20250419220207
[EXTRACT] PACQUI_20250420220208.zip -> PACQUI_20250420220208
[EXTRACT] PACQUI_20250421220209.zip -> PACQUI_20250421220209
[EXTRACT] PACQUI_20250422220210.zip -> PACQUI_20250422220210
[EXTRACT] PACQUI_20250423220211.zip -> PACQUI_20250423220211
[EXTRACT] PACQUI_20250424220212.zip -> PACQUI_20250424220212
[EXTRACT] PACQUI_20250425220213.zip -> PACQUI_20250425220213
[EXTRACT] PACQUI_20250426220214.zip -> PACQUI_20250426220214
[EXTRACT] PACQUI_20250427220215.zip -> PACQUI_20250427220215
[EXTRACT] PACQUI_20250428220216.zip -> PACQUI_20250428220216
[EXTRACT] PACQUI_20250429220217.zip -> PACQUI_20250429220217
[EXTRACT] PACQUI_20250430220218.zip -> PACQUI_20250430220218
[EXTRACT] PACQUI_20250501220219.zip -> PACQUI_20250501220219
[EXTRACT] PACQUI_20250502220220.zip -> PACQUI_20250502220220
[EXTRACT] PACQUI_20250503220221.zip -> PACQUI_20250503220221
[EXTRACT] PACQUI_2025050

## 🗂️ 3. List All `.25O` Files
Lists all `.25O` files consolidated in the PACQUI folder for verification.

In [3]:
from pathlib import Path

# Folder where all PACQUI .25O files were copied
root = Path(r"C:\Users\varga\Box\Data\nc Files\PACQUI")

# Find all .25O files
files = list(root.glob("*.25O"))

print("Found", len(files), "files:")
for f in files:
    print(f.name)

Found 132 files:
PACQUI_raw_20250413171438.25O
PACQUI_raw_20250414171440.25O
PACQUI_raw_20250415171441.25O
PACQUI_raw_20250415215413.25O
PACQUI_raw_20250415220203.25O
PACQUI_raw_20250416220203.25O
PACQUI_raw_20250417220205.25O
PACQUI_raw_20250418220206.25O
PACQUI_raw_20250419220207.25O
PACQUI_raw_20250420220208.25O
PACQUI_raw_20250421220209.25O
PACQUI_raw_20250422220210.25O
PACQUI_raw_20250423220211.25O
PACQUI_raw_20250424220212.25O
PACQUI_raw_20250425220213.25O
PACQUI_raw_20250426220214.25O
PACQUI_raw_20250427220215.25O
PACQUI_raw_20250428220216.25O
PACQUI_raw_20250429220217.25O
PACQUI_raw_20250430220218.25O
PACQUI_raw_20250501220219.25O
PACQUI_raw_20250502220220.25O
PACQUI_raw_20250503220221.25O
PACQUI_raw_20250504220222.25O
PACQUI_raw_20250506142454.25O
PACQUI_raw_20250507142455.25O
PACQUI_raw_20250508142456.25O
PACQUI_raw_20250509142457.25O
PACQUI_raw_20250510142458.25O
PACQUI_raw_20250511142459.25O
PACQUI_raw_20250512142501.25O
PACQUI_raw_20250513142501.25O
PACQUI_raw_202505141425

## ⚙️ 4. Convert `.25O` Files to NetCDF
Processes `.25O` observation files into NetCDF format using `gnssvod.preprocess()`.
- Creates subfolders `processed/` (output) and `aux_orbits/` (for SP3/CLK cache).
- Skips files already processed unless `FORCE_REDO=True`.
- Reports skipped, successful, and failed files.

In [4]:
from pathlib import Path
import glob
import os
import xarray as xr
import gnssvod as gv

# ==== Configuration ====
FORCE_REDO = False  # set True to reprocess even if output exists

# Folder containing all PACQUI .25O files
DEST = Path(r"C:\Users\varga\Box\Data\nc Files\PACQUI")

# 1) Input RINEX file list
rinex_files = sorted(glob.glob(str(DEST / '*.25O')))
print(f'Found {len(rinex_files)} .25O files (showing up to 5):', rinex_files[:5])
if len(rinex_files) == 0:
    raise FileNotFoundError("No .25O files found in DEST folder. Check DEST and file existence.")

# 2) Output folder for NetCDF files
outdir = DEST / "processed"
outdir.mkdir(parents=True, exist_ok=True)

# 3) Persistent folder for orbits/clocks
aux_dir = DEST / "aux_orbits"
aux_dir.mkdir(parents=True, exist_ok=True)

# 4) Variables to keep (SNR)
keepvars = ['S?', 'S??']

# ---- Utility functions to decide whether processing is needed ----
def has_output_for(stem: str) -> bool:
    nc_hits = list(outdir.glob(f"{stem}*.nc"))
    zarr_hits = list(outdir.glob(f"{stem}*.zarr"))
    return (len(nc_hits) + len(zarr_hits)) > 0

def needs_processing(rinex_path: str) -> bool:
    if FORCE_REDO:
        return True
    stem = Path(rinex_path).stem
    return not has_output_for(stem)

ok, bad, skipped = [], [], []

# 5) Process each file with error handling
for rinex in rinex_files:
    stem = Path(rinex).stem

    if not needs_processing(rinex):
        print(f"[SKIP] {stem} (output already exists in '{outdir.name}')")
        skipped.append(rinex)
        continue

    try:
        # Pattern and output for this file
        pattern = {'PACQUI': rinex}
        outputdir = {'PACQUI': str(outdir)}

        result = gv.preprocess(
            pattern,
            interval='15s',
            keepvars=keepvars,
            outputdir=outputdir,
            outputresult=True,
            aux_path=str(aux_dir)
        )

        # Basic validation of the result
        pacqui_res = result.get('PACQUI', [])
        if isinstance(pacqui_res, list):
            if not pacqui_res:
                raise ValueError(f"No dataset produced for: {rinex}")
            _ = pacqui_res[0]
        elif isinstance(pacqui_res, xr.Dataset):
            _ = pacqui_res
        else:
            raise TypeError(f"Unexpected return type for 'PACQUI': {type(pacqui_res)}")

        print(f"[OK] {Path(rinex).name}")
        ok.append(rinex)

    except Exception as e:
        print(f"[WARN] Skipping due to error: {Path(rinex).name}")
        print("       ", e)
        bad.append((rinex, str(e)))
        continue

# 6) Final summary
print("\n===== SUMMARY =====")
print(f"Processed OK: {len(ok)}")
print(f"Skipped    : {len(skipped)} (already had output)")
print(f"Failed     : {len(bad)}")

if bad:
    print("\nFiles that failed:")
    for fname, err in bad:
        print(f" - {fname}: {err}")
else:
    print("\nAll files processed successfully! 🤙")

Found 132 .25O files (showing up to 5): ['C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\PACQUI_raw_20250413171438.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\PACQUI_raw_20250414171440.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\PACQUI_raw_20250415171441.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\PACQUI_raw_20250415215413.25O', 'C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\PACQUI_raw_20250415220203.25O']
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\PACQUI\processed*.nc
C:\Users\varga\Box\Data\nc Files\PACQUI\PACQUI_raw_20250413171438.25O exists | Reading...
Processed 0 out of 3201676 lines (0.0%)
Processed 96879 out of 3201676 lines (3.0%)
Processed 492748 out of 3201676 lines (15.4%)
Processed 950904 out of 3201676 lines (29.7%)
Processed 1092446 out of 3201676 lines (34.1%)
Processed 1561059 out of 3201676 lines (48.8%)
Processed 1644822 out of 3201676 lines (51.4%)
Processed 1865708 out of 3201676 lines (58.3%)
Proce

GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3.gz: 1.03MB [00:07, 147kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3.gz
C:\Users\varga\Box\Data\nc Files\PACQUI\aux_orbits\GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3 file is read in 15.81 seconds
This file does not exist: C:\Users\varga\Box\Data\nc Files\PACQUI\aux_orbits\GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3.gz: 1.03MB [00:03, 316kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3.gz
C:\Users\varga\Box\Data\nc Files\PACQUI\aux_orbits\GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3 file is read in 10.62 seconds
This file does not exist: C:\Users\varga\Box\Data\nc Files\PACQUI\aux_orbits\GFZ0MGXRAP_20251030000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251030000_01D_30S_CLK.CLK.gz

...

 | Download completed for GFZ0MGXRAP_20252340000_01D_30S_CLK.CLK.gz
C:\Users\varga\Box\Data\nc Files\PACQUI\aux_orbits\GFZ0MGXRAP_20252340000_01D_30S_CLK.CLK file is read in 2.20 seconds
SP3 interpolation is done in 3.05 seconds
Saved 98862 individual observations in C:\Users\varga\Box\Data\nc Files\PACQUI\processed\PACQUI_raw_20250821210803.nc
[OK] PACQUI_raw_20250821210803.25O
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\PACQUI\processed*.nc
C:\Users\varga\Box\Data\nc Files\PACQUI\PACQUI_raw_20250822210804.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\nc Files\PACQUI\PACQUI_raw_20250822210804.25O  is read in 3.14 seconds.
Processing 97577 individual observations
Calculating Azimuth and Elevation
C:\Users\varga\Box\Data\nc Files\PACQUI\aux_orbits\GFZ0MGXRAP_20252340000_01D_05M_ORB.SP3 exists | Reading...
C:\Users\varga\Box\Data\nc Files\PACQUI\aux_orbits\GFZ0MGXRAP_20252340000_01D_05M_ORB.SP3 file is read in 0.64 seconds
This file 

GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz: 0.00B [00:02, ?B/s]


<urlopen error <urlopen error ftp error: error_perm('550 CWD command failed: directory not found.')>>
 | Requested file ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz cannot be not found!
[WARN] Skipping due to error: PACQUI_raw_20250822210804.25O
        [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'
Could not find any files matching the pattern C:\Users\varga\Box\Data\nc Files\PACQUI\processed*.nc
C:\Users\varga\Box\Data\nc Files\PACQUI\PACQUI_raw_20250823210805.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\nc Files\PACQUI\PACQUI_raw_20250823210805.25O  is read in 0.18 seconds.
Processing 6144 individual observations
Calculating Azimuth and Elevation
This file does not exist: C:\Users\varga\Box\Data\nc Files\PACQUI\aux_orbits\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D_

GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz: 0.00B [00:02, ?B/s]

<urlopen error <urlopen error ftp error: error_perm('550 CWD command failed: directory not found.')>>
 | Requested file ftp://gssc.esa.int/gnss/products/2380//GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3.gz cannot be not found!
[WARN] Skipping due to error: PACQUI_raw_20250823210805.25O
        [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'

===== SUMMARY =====
Processed OK: 130
Skipped    : 0 (already had output)
Failed     : 2

Files that failed:
 - C:\Users\varga\Box\Data\nc Files\PACQUI\PACQUI_raw_20250822210804.25O: [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'
 - C:\Users\varga\Box\Data\nc Files\PACQUI\PACQUI_raw_20250823210805.25O: [Errno 2] No such file or directory: 'C:\\Users\\varga\\Box\\Data\\nc Files\\PACQUI\\aux_orbits\\GFZ0MGXRAP_20252350000_01D_05M_ORB.SP3'
